In [ ]:
!pip install undetected-chromedriver selenium_stealth 

In [42]:
import undetected_chromedriver as uc


from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import time

# Set up the driver (this assumes you have chromedriver installed and in your PATH)
driver = uc.Chrome()

# Get the webpage
driver.get("https://www.sahibinden.com/kategori-vitrin?viewType=Classic&pagingSize=50&category=3530&sorting=price_asc")

# Wait for the JavaScript to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
time.sleep(5)

# Parse the page with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find the table on the page
table = soup.find_all('table')[0] 

# Get the table headers
headers = [th.text for th in table.find_all('th')]

# Get the table rows
rows = table.find_all('tr')

# Get the data from each row
data = []
for row in rows[1:]:  # Skip the header row
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])  # Get rid of empty values

data


[['Tofaş',
  'Şahin',
  '1.4',
  'muayeneli yeni motor sorunsuz',
  '1985',
  '210.000',
  'Beyaz',
  '59.500 TL',
  '18 Mayıs\n\n2023',
  'TekirdağSüleymanpaşa'],
 ['Volkswagen',
  'Passat',
  '1.6 TD GL',
  'Passat 1.6 TDI',
  '1990',
  '101.010',
  'Lacivert',
  '65.000 TL',
  '22 Mayıs\n\n2023',
  'MuğlaOrtaca'],
 ['Tofaş',
  'Murat',
  '131',
  'ORJİNAL MAVİ BONCUK 131 ŞAHİN SIFIR MUAYENELİ HİÇ TÜP GÖRMEMİŞ',
  '1978',
  '150.000',
  'Mavi',
  '65.750 TL',
  '22 Mayıs\n\n2023',
  'KütahyaTavşanlı'],
 [],
 ['Tofaş',
  'Serçe',
  'Serçe',
  'Sahibinden Temiz Takaslı Piyasaya İnat Fiyat Yükselmeyecek',
  '1989',
  '200.000',
  'Bej',
  '67.499 TL',
  '22 Mayıs\n\n2023',
  'AnkaraÇankaya'],
 ['Tofaş',
  'Doğan',
  '1.6',
  'DOĞAN 5 İLERİ',
  '1987',
  '1.111',
  'Kırmızı',
  '67.500 TL',
  '21 Mayıs\n\n2023',
  'ManisaSoma'],
 ['Skoda',
  'Favorit',
  '136 L',
  'Sahibinden dededen',
  '1991',
  '64.000',
  'Kırmızı',
  '74.000 TL',
  '23 Mayıs\n\n2023',
  'İstanbulÇekmeköy'],
 ['Tofa

In [56]:
import dataclasses
from dataclasses import dataclass, field

@dataclass
class CarListing:
    brand: str
    series: str
    model: str
    ad_title: str
    year: int
    mileage: int
    color: str
    price: float
    ad_date: str
    city: str
    district: str
    url: str

    def __post_init__(self):
        for field_name in self.__dataclass_fields__:
            field_value = getattr(self, field_name)
            if field_value == dataclasses.MISSING:
                raise ValueError(f"{field_name} is required.")


def parse_row(row:bs4.Tag):

    # Extracting the data from the HTML
    # Extracting the data from the HTML
    marka = row.find('td', class_='searchResultsTagAttributeValue').text.strip()
    seri = row.find('td', class_='searchResultsTagAttributeValue').find_next('td').text.strip()
    model = row.find('td', class_='searchResultsTagAttributeValue').find_next('td').find_next('td').text.strip()
    ilan_basligi = row.find('a', class_='classifiedTitle').text.strip()
    url = row.find('a', class_='classifiedTitle').get('href')
    yil = int(row.find('td', class_='searchResultsAttributeValue').text.strip())
    km = int(row.find('td', class_='searchResultsAttributeValue').find_next('td').text.strip().replace('.', ''))
    renk = row.find('td', class_='searchResultsAttributeValue').find_next('td').find_next('td').text.strip()
    fiyat = float(row.find('td', class_='searchResultsPriceValue').text.strip().split()[0].replace('.', ''))
    ilan_tarihi = row.find('td', class_='searchResultsDateValue').find('span').text.strip()
    il = row.find('td', class_='searchResultsLocationValue').contents[0].text.strip()
    ilce = row.find('td', class_='searchResultsLocationValue').contents[2].text.strip()

    # Creating a CarListing instance
    car = CarListing(
        brand=marka,
        series=seri,
        model=model,
        ad_title=ilan_basligi,
        year=yil,
        mileage=km,
        color=renk,
        price=fiyat,
        ad_date=ilan_tarihi,
        city=il,
        district=ilce,
        url=url
    )
    return car

In [55]:
rowText = """
<tr
  data-id="1092327047"
  class="searchResultsItem searchResultsPromoHighlight searchResultsPromoBold"
>
  <td class="searchResultsLargeThumbnail" data-hj-suppress="">
    <a
      href="/ilan/vasita-otomobil-tofas-muayeneli-yeni-motor-sorunsuz-1092327047/detay"
      title="muayeneli yeni motor sorunsuz"
    >
      <img
        class=""
        src="https://i0.shbdn.com/photos/32/70/47/lthmb_10923270479nn.jpg"
        alt="muayeneli yeni motor sorunsuz #1092327047"
        title="muayeneli yeni motor sorunsuz"
      />
    </a>
  </td>
  <td class="searchResultsTagAttributeValue">Tofaş</td>
  <td class="searchResultsTagAttributeValue">Şahin</td>
  <td class="searchResultsTagAttributeValue">1.4</td>
  <td class="searchResultsTitleValue mini-column">
    <input id="favoriteClassifiedsVisibility" type="hidden" value="true" />
    <div class="action-wrapper" data-classified-id="1092327047">
      <div class="add-to-favorites last favorite">
        <a
          href="#"
          class="action classifiedAddFavorite trackClick trackId_favorite hidden"
          data-content="Favorilerime Ekle"
        >
        </a>
        <a
          href="#"
          class="action classifiedRemoveFavorite trackClick trackId_favorite disable"
          data-content="Favorilerimde"
        >
        </a>
      </div>
      <div class="compare hidden">
        <a
          class="facetedCheckbox action compare-classified"
          data-content="İlan Karşılaştır"
        >
          <i></i>
        </a>
      </div>
    </div>
    <a
      class="classifiedTitle"
      title="muayeneli yeni motor sorunsuz"
      href="/ilan/vasita-otomobil-tofas-muayeneli-yeni-motor-sorunsuz-1092327047/detay"
    >
      muayeneli yeni motor sorunsuz</a
    >

    <img
      class="titleIcon"
      alt="Videolu İlan"
      title="Videolu İlan"
      width="15"
      height="15"
      src="https://s0.shbdn.com/assets/images/iconHasVideo@2x:4667ff5b87098416c7b8e2219e70ff40.png"
      style="visibility: visible"
    />
    <!-- This file has been included to desktop(classic, list, gallery view) and responsive -->
  </td>
  <td class="searchResultsAttributeValue">1985</td>
  <td class="searchResultsAttributeValue">210.000</td>
  <td class="searchResultsAttributeValue">Beyaz</td>
  <td class="searchResultsPriceValue">
    <div class="classified-price-container">
      <span> 59.500 TL</span>
    </div>

    <input id="584799372" type="hidden" value="[1, 10, 4]" />
  </td>
  <td class="searchResultsDateValue">
    <span>18 Mayıs</span>
    <br />
    <span>2023</span>
  </td>
  <td class="searchResultsLocationValue">Tekirdağ<br />Süleymanpaşa</td>
  <td class="ignore-me">
    <a
      href="#"
      class="mark-as-ignored"
      title="Bu ilanla ilgilenmiyorum, gizle."
    ></a>
    <a href="#" class="mark-as-not-ignored disable" data-content="Göster"></a>
  </td>
</tr>
"""

msoup = BeautifulSoup(rowText, 'html.parser')

# Extracting the data from the HTML
marka = msoup.find('td', class_='searchResultsTagAttributeValue').text.strip()
seri = msoup.find('td', class_='searchResultsTagAttributeValue').find_next('td').text.strip()
model = msoup.find('td', class_='searchResultsTagAttributeValue').find_next('td').find_next('td').text.strip()
ilan_basligi = msoup.find('a', class_='classifiedTitle').text.strip()
url = msoup.find('a', class_='classifiedTitle').get('href')
yil = int(msoup.find('td', class_='searchResultsAttributeValue').text.strip())
km = int(msoup.find('td', class_='searchResultsAttributeValue').find_next('td').text.strip().replace('.', ''))
renk = msoup.find('td', class_='searchResultsAttributeValue').find_next('td').find_next('td').text.strip()
fiyat = float(msoup.find('td', class_='searchResultsPriceValue').text.strip().split()[0].replace('.', ''))
ilan_tarihi = msoup.find('td', class_='searchResultsDateValue').find('span').text.strip()
il = msoup.find('td', class_='searchResultsLocationValue').contents[0].text.strip()
ilce = msoup.find('td', class_='searchResultsLocationValue').contents[2].text.strip()

In [57]:
# Parse the data from soup.

cars = []

table = soup.find("tbody", {"class": "searchResultsRowClass"})

# Find all the rows with the specified class
rows_to_remove = table.find_all('tr', class_='searchResultsItem nativeAd classicNativeAd')

# Remove each row from the table
for row in rows_to_remove:
    row:bs4.Tag
    row.extract()

rows = table.find_all("tr", {"class": "searchResultsItem"})

for row in rows:
    try:
        car = parse_row(row)
    except Exception as e:
        print(row)
        raise e
    print(car)
    cars.append(car)




CarListing(brand='Tofaş', series='Şahin', model='1.4', ad_title='muayeneli yeni motor sorunsuz', year=1985, mileage=210000, color='Beyaz', price=59500.0, ad_date='18 Mayıs', city='Tekirdağ', district='Süleymanpaşa', url='/ilan/vasita-otomobil-tofas-muayeneli-yeni-motor-sorunsuz-1092327047/detay')
CarListing(brand='Volkswagen', series='Passat', model='1.6 TD GL', ad_title='Passat 1.6 TDI', year=1990, mileage=101010, color='Lacivert', price=65000.0, ad_date='22 Mayıs', city='Muğla', district='Ortaca', url='/ilan/vasita-otomobil-volkswagen-passat-1.6-tdi-1101071229/detay')
CarListing(brand='Tofaş', series='Murat', model='131', ad_title='ORJİNAL MAVİ BONCUK 131 ŞAHİN SIFIR MUAYENELİ HİÇ TÜP GÖRMEMİŞ', year=1978, mileage=150000, color='Mavi', price=65750.0, ad_date='22 Mayıs', city='Kütahya', district='Tavşanlı', url='/ilan/vasita-otomobil-tofas-orjinal-mavi-boncuk-131-sahin-sifir-muayeneli-hic-tup-gormemis-1101078619/detay')
CarListing(brand='Tofaş', series='Serçe', model='Serçe', ad_title

In [48]:
cars

[CarListing(brand='Tofaş', series='Şahin', model='1.4', ad_title='muayeneli yeni motor sorunsuz', year=1985, mileage=210000, color='Beyaz', price=59500.0, ad_date='18 Mayıs', city='Tekirdağ', district='Süleymanpaşa', url=''),
 CarListing(brand='Volkswagen', series='Passat', model='1.6 TD GL', ad_title='Passat 1.6 TDI', year=1990, mileage=101010, color='Lacivert', price=65000.0, ad_date='22 Mayıs', city='Muğla', district='Ortaca', url=''),
 CarListing(brand='Tofaş', series='Murat', model='131', ad_title='ORJİNAL MAVİ BONCUK 131 ŞAHİN SIFIR MUAYENELİ HİÇ TÜP GÖRMEMİŞ', year=1978, mileage=150000, color='Mavi', price=65750.0, ad_date='22 Mayıs', city='Kütahya', district='Tavşanlı', url='')]

In [83]:

brands_ul_text= """
<ul>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3545&amp;sorting=price_asc" title="Alfa Romeo"><h2>Alfa Romeo</h2></a>
        <span>(24)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3549&amp;sorting=price_asc" title="Audi"><h2>Audi</h2></a>
        <span>(531)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3551&amp;sorting=price_asc" title="BMW"><h2>BMW</h2></a>
        <span>(995)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=23271&amp;sorting=price_asc" title="Chery"><h2>Chery</h2></a>
        <span>(1)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3555&amp;sorting=price_asc" title="Chevrolet"><h2>Chevrolet</h2></a>
        <span>(73)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3556&amp;sorting=price_asc" title="Chrysler"><h2>Chrysler</h2></a>
        <span>(7)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3557&amp;sorting=price_asc" title="Citroën"><h2>Citroën</h2></a>
        <span>(206)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=246122&amp;sorting=price_asc" title="Cupra"><h2>Cupra</h2></a>
        <span>(7)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3559&amp;sorting=price_asc" title="Dacia"><h2>Dacia</h2></a>
        <span>(86)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3561&amp;sorting=price_asc" title="Daihatsu"><h2>Daihatsu</h2></a>
        <span>(2)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=233138&amp;sorting=price_asc" title="DS Automobiles"><h2>DS Automobiles</h2></a>
        <span>(9)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3564&amp;sorting=price_asc" title="Ferrari"><h2>Ferrari</h2></a>
        <span>(1)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3607&amp;sorting=price_asc" title="Fiat"><h2>Fiat</h2></a>
        <span>(858)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3565&amp;sorting=price_asc" title="Ford"><h2>Ford</h2></a>
        <span>(558)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=90019&amp;sorting=price_asc" title="Geely"><h2>Geely</h2></a>
        <span>(4)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3567&amp;sorting=price_asc" title="Honda"><h2>Honda</h2></a>
        <span>(436)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3568&amp;sorting=price_asc" title="Hyundai"><h2>Hyundai</h2></a>
        <span>(482)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=89919&amp;sorting=price_asc" title="Infiniti"><h2>Infiniti</h2></a>
        <span>(1)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3571&amp;sorting=price_asc" title="Jaguar"><h2>Jaguar</h2></a>
        <span>(10)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3572&amp;sorting=price_asc" title="Kia"><h2>Kia</h2></a>
        <span>(90)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3573&amp;sorting=price_asc" title="Lada"><h2>Lada</h2></a>
        <span>(8)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3575&amp;sorting=price_asc" title="Lancia"><h2>Lancia</h2></a>
        <span>(3)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3576&amp;sorting=price_asc" title="Lexus"><h2>Lexus</h2></a>
        <span>(1)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3579&amp;sorting=price_asc" title="Maserati"><h2>Maserati</h2></a>
        <span>(3)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3580&amp;sorting=price_asc" title="Mazda"><h2>Mazda</h2></a>
        <span>(27)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3581&amp;sorting=price_asc" title="Mercedes - Benz"><h2>Mercedes - Benz</h2></a>
        <span>(733)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=64227&amp;sorting=price_asc" title="MG"><h2>MG</h2></a>
        <span>(1)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3584&amp;sorting=price_asc" title="Mini"><h2>Mini</h2></a>
        <span>(46)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3585&amp;sorting=price_asc" title="Mitsubishi"><h2>Mitsubishi</h2></a>
        <span>(12)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3588&amp;sorting=price_asc" title="Nissan"><h2>Nissan</h2></a>
        <span>(43)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3591&amp;sorting=price_asc" title="Opel"><h2>Opel</h2></a>
        <span>(750)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3592&amp;sorting=price_asc" title="Peugeot"><h2>Peugeot</h2></a>
        <span>(412)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3595&amp;sorting=price_asc" title="Porsche"><h2>Porsche</h2></a>
        <span>(25)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=250043&amp;sorting=price_asc" title="Regal Raptor"><h2>Regal Raptor</h2></a>
        <span>(1)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3597&amp;sorting=price_asc" title="Renault"><h2>Renault</h2></a>
        <span>(1.635)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=248079&amp;sorting=price_asc" title="RKS"><h2>RKS</h2></a>
        <span>(1)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3598&amp;sorting=price_asc" title="Rolls-Royce"><h2>Rolls-Royce</h2></a>
        <span>(2)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3599&amp;sorting=price_asc" title="Rover"><h2>Rover</h2></a>
        <span>(4)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3600&amp;sorting=price_asc" title="Saab"><h2>Saab</h2></a>
        <span>(2)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3601&amp;sorting=price_asc" title="Seat"><h2>Seat</h2></a>
        <span>(252)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3602&amp;sorting=price_asc" title="Skoda"><h2>Skoda</h2></a>
        <span>(258)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3603&amp;sorting=price_asc" title="Smart"><h2>Smart</h2></a>
        <span>(3)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3604&amp;sorting=price_asc" title="Subaru"><h2>Subaru</h2></a>
        <span>(3)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3605&amp;sorting=price_asc" title="Suzuki"><h2>Suzuki</h2></a>
        <span>(13)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3606&amp;sorting=price_asc" title="Tata"><h2>Tata</h2></a>
        <span>(5)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=236720&amp;sorting=price_asc" title="Tesla"><h2>Tesla</h2></a>
        <span>(2)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=113644&amp;sorting=price_asc" title="Tofaş"><h2>Tofaş</h2></a>
        <span>(237)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3608&amp;sorting=price_asc" title="Toyota"><h2>Toyota</h2></a>
        <span>(395)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3611&amp;sorting=price_asc" title="Volkswagen"><h2>Volkswagen</h2></a>
        <span>(1.688)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=249751&amp;sorting=price_asc" title="Volta"><h2>Volta</h2></a>
        <span>(2)</span>
    </li>
<li class="cl2">
        <a href="/kategori-vitrin?viewType=Classic&amp;pagingSize=50&amp;category=3612&amp;sorting=price_asc" title="Volvo"><h2>Volvo</h2></a>
        <span>(83)</span>
    </li>
</ul>
"""

import re

# Define the regular expression pattern
category_pattern = r"category=(\d+)"

brands_ul = BeautifulSoup(
    brands_ul_text, "html.parser")


brands_list = []

# We will get every href in each li in brands_ul

lis = brands_ul.find_all("li")
for li in lis:
    li: bs4.Tag
    a = li.find("a", href=True)
    span = li.find("span")

    # get relevant data
    brand_name = a.text.strip()
    href = a["href"].strip()
    
    # get count as int 
    count = int(span.text.replace("(", "").replace(")", "").replace(".", ""))

    # Use re.search to find the first occurrence of the pattern in the URL
    match = re.search(category_pattern, href)
    category = int(match.group(1)) if match is not None else None

    # Create a dictionary with the data
    brand_data = {
    "name": brand_name,
    "href": href,
    "count": count,
    "category": category
    }

    # Append the dictionary to the brands list
    brands_list.append(brand_data)



brands_list


[{'name': 'Alfa Romeo',
  'href': '/kategori-vitrin?viewType=Classic&pagingSize=50&category=3545&sorting=price_asc',
  'count': 24,
  'category': 3545},
 {'name': 'Audi',
  'href': '/kategori-vitrin?viewType=Classic&pagingSize=50&category=3549&sorting=price_asc',
  'count': 531,
  'category': 3549},
 {'name': 'BMW',
  'href': '/kategori-vitrin?viewType=Classic&pagingSize=50&category=3551&sorting=price_asc',
  'count': 995,
  'category': 3551},
 {'name': 'Chery',
  'href': '/kategori-vitrin?viewType=Classic&pagingSize=50&category=23271&sorting=price_asc',
  'count': 1,
  'category': 23271},
 {'name': 'Chevrolet',
  'href': '/kategori-vitrin?viewType=Classic&pagingSize=50&category=3555&sorting=price_asc',
  'count': 73,
  'category': 3555},
 {'name': 'Chrysler',
  'href': '/kategori-vitrin?viewType=Classic&pagingSize=50&category=3556&sorting=price_asc',
  'count': 7,
  'category': 3556},
 {'name': 'Citroën',
  'href': '/kategori-vitrin?viewType=Classic&pagingSize=50&category=3557&sorting

In [79]:
len(brand_urls)


51